In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from itertools import product
import multiprocessing
from math import sqrt,log,exp

In [2]:
def pos(i,prod,L):
    '''
    Compute positive function and gradient information
    
    input:
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fpt - positive function value
        gfpt - positive function gradient
    '''
    plus = L/2+prod
    fpt = plus**i
    gfpt = fpt*i/plus # no xt yet!
    
    # hfpt = i*(i-1)*(L/2+prod)**(i-2)
    return fpt,gfpt              

In [3]:
def comb(n, k):
    '''
    Compute combination
    
    input:
        n - total number
        k - number of chosen
    
    output:
        c - number of combination
    '''
    return factorial(n) / factorial(k) / factorial(n - k)

In [4]:
comb_dict = {0:{0:1},1:{0:1,1:1},2:{0:1,1:2,2:1},3:{0:1,1:3,2:3,3:1},4:{0:1,1:4,2:6,3:4,4:1},
             5:{0:1,1:5,2:10,3:10,4:5,5:1},6:{0:1,1:6,2:15,3:20,4:15,5:6,6:1},
             7:{0:1,1:7,2:21,3:35,4:35,5:21,6:7,7:1},8:{0:1,1:8,2:28,3:56,4:70,5:56,6:28,7:8,8:1},
             9:{0:1,1:9,2:36,3:84,4:126,5:126,6:84,7:36,8:9,9:1},
             10:{0:1,1:10,2:45,3:120,4:210,5:252,6:210,7:120,8:45,9:10,10:1}}

In [5]:
beta_dict = {0:np.array([11,110,495,1320,2310,2772,2310,1320,495,110,11]),
             1:np.array([110,990,3960,9240,13860,13860,9240,3960,990,110]),
             2:np.array([495,3960,13860,27720,34650,27720,13860,3960,495]),
             3:np.array([1320,9240,27720,46200,46200,27720,9240,1320]),
             4:np.array([2310,13860,34650,46200,34650,13860,2310]),5:np.array([2772,13860,27720,27720,13860,2772]),
             6:np.array([2310,9240,13860,9240,2310]),7:np.array([1320,3960,3960,1320]),8:np.array([495,990,495]),
             9:np.array([110,110]),10:np.array([11])}

In [6]:
def neg(loss,i,prod,L):
    '''
    Compute negative function and gradient information
    
    input:
        loss - loss function
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fnt - negative function value
        gfnt - negative function gradient
    '''
    minus = L/2-prod
    #FNT = np.zeros(N+1-i)
    #GFNT = np.zeros(N+1-i)
    fnt = 0.0
    gfnt = 0.0
    # hfnt = 0.0
    for k in range(i,N+1):
        # compute forward difference
        delta = 0.0
        for j in range(k+1):
            delta += comb_dict[k][j]*(-1)**(k-j)*loss(j/N)
            
        # compute coefficient
        #beta = beta_dict[i][k-i]*delta/(2*L)**k*minus**(k-i)
        beta = comb_dict[N][k]*comb_dict[k][i]*(N+1)*delta/(2*L)**k*minus**(k-i)
        # compute function value
        fnt += beta
        # compute gradient
        gfnt += beta*(k-i)/minus  # no xt yet!
        '''
        # compute temp function value
        FNT[k-i] = delta/(2*L)**k*(minus**(k-i))
        GFNT[k-i] = FNT[k-i]*(k-i)/minus
    # compute function value
    fnt = np.dot(beta_dict[i],FNT)
    # compute gradient
    gfnt = np.dot(beta_dict[i],GFNT) # no xt yet!
    '''
        # compute hessian
        # hfnt += beta*(k-i)*(k-i-1)*(L/2-prod)**(k-i-2)
    return fnt,gfnt

In [7]:
def w_grad(gfpt,gfnt,yt,at,bt,alphat):
    '''
    Gradient with respect to w
    
    input:
        fpt - positive function at t
        gfpt - positive function gradient at t
        fnt - negative function at t
        gfnt - negative function gradient at t
        yt - sample label at t
        pt - p at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        gradwt - gradient w.r.t. w at t
    '''
    if yt == 1:
        gradwt = 2*(alphat - at)*gfpt
    else:
        gradwt = 2*(alphat - bt)*gfnt
    return gradwt

In [8]:
def w_hess(hfpt,hfnt,yt,at,bt,alphat):
    hesswt = 0.0
    if yt == 1:
        hesswt = 2*(alphat - at)*hfpt
    else:
        hesswt = 2*(alphat - bt)*hfnt
    return hesswt

In [9]:
def proj(wt,R):
    '''
    Projection
    
    input:
        wt - w at t
        R - radius
        
    output:
        proj - projected wt
    '''
    norm = np.linalg.norm(wt)
    if norm > R:
        wt = wt/norm*R
    return wt

In [10]:
def a_grad(fpt,yt,at):
    '''
    Gradient with respect to a
    
    input:
        fpt - positive function at t
        yt - sample label at t
        pt - p at t
        at - a at t
    
    output:
        gradat - gradient w.r.t a at t
    '''
    gradat = 0.0 
    if yt == 1:
        gradat = 2*(at - fpt)
    else:
        gradat = 2*at
    return gradat

In [11]:
def b_grad(fnt,yt,bt):
    '''
    Gradient with respect to b
    
    input:
        fnt - negative function at t
        yt - sample label at t
        pt - p at t
        bt - b at t
    
    output:
        gradbt - gradient w.r.t b at t
    '''
    gradbt = 0.0 
    if yt == 1:
        gradbt = 2*bt
    else:
        gradbt = 2*(bt - fnt)
    return gradbt

In [12]:
def alpha_grad(fpt,fnt,yt,alphat):
    '''
    Gradient with respect to alpha
    '''
    gradalphat = 0.0
    if yt == 1:
        gradalphat = -2*(alphat - fpt)
    else:
        gradalphat = -2*(alphat - fnt)
    return gradalphat

In [13]:
def loader(filename):
    '''
    Data file loader
    
    input:
        filename - filename
    
    output:
        x - sample features
        y - sample labels
    '''
    # raw data
    L = []
    with open(filename,'r') as file:
        for line in csv.reader(file, delimiter = ' '):
            line[0] = '0:'+line[0]
            line.remove('')
            L.append(dict(i.split(':') for i in line))
    df = pd.DataFrame(L,dtype=float).fillna(0)
    X = df.iloc[:,1:].values
    Y = df.iloc[:,0].values
    # centralize
    mean = np.mean(X,axis=1)
    X = (X.transpose() - mean).transpose()
    # normalize
    norm = np.linalg.norm(X,axis=1)
    X = X/norm[:,None]
    # convert to binary class
    r = np.ptp(Y)
    index = np.argwhere(Y<r//2)
    INDEX = np.argwhere(Y>=r//2)
    Y[index] = -1
    Y[INDEX] = 1
    Y = Y.astype(int)
    return X,Y

In [14]:
def split(FEATURES,LABELS,folder,folders):
    
    if folder > folders:
        print('Exceed maximum folders!')
        return
    # load and split data
    #FEATURES,LABELS = loader(dataset)
    n,d = FEATURES.shape
    # regular portion of each folder
    portion = round(n/folders)
    start = portion*folder
    stop = portion*(folder+1)
    if np.abs(stop - n) < portion: # remainder occurs
        X_train = FEATURES[:start,:]
        Y_train = LABELS[:start]
        X_test = FEATURES[start:,:]
        Y_test = LABELS[start:]
    else:
        mask = np.ones(n, bool)
        mask[start:stop] = False
        X_train = FEATURES[mask,:]
        Y_train = LABELS[mask]
        X_test = FEATURES[start:stop]
        Y_test = LABELS[start:stop]
    # get dimensions of the data
    # n,_ = X_train.shape
    # number of epoch
    # epoch = T//n+1
    # augment by epoch
    # X_train_augmented = np.tile(X_train,(epoch,1)) # might have memory burden
    # Y_train_augmented = np.tile(Y_train,epoch)
    
    #return X_train_augmented,X_test,Y_train_augmented,Y_test
    return X_train,X_test,Y_train,Y_test

In [15]:
def prox(eta,loss,x,y,L,gamma,lam,wj,aj,bj,alphaj,bwt,bat,bbt,balphat):
    '''
    perform proximal guided gradient descent when receive an sample
    '''
    prod = np.dot(wj,x)
    fpt = np.zeros(N+1)
    gfpt = np.zeros(N+1)
    # hfpt = np.zeros(N+1)
    fnt = np.zeros(N+1)
    gfnt = np.zeros(N+1)
    # hfnt = np.zeros(N+1)
    gradwt = 0.0
    gradat = 0.0
    gradbt = 0.0
    gradalphat = 0.0
    # hesswt = 0.0
    for i in range(N+1):
        fpt[i],gfpt[i] = pos(i,prod,L)
        #print(fpt[i])
        fnt[i],gfnt[i] = neg(loss,i,prod,L)
        gradwt += w_grad(gfpt[i],gfnt[i],y,aj[i],bj[i],alphaj[i])# accumulate i
        # hesswt += w_hess(hfpt[i],hfnt[i],y,aj[i],bj[i],alphaj[i])
        gradat = a_grad(fpt[i],y,aj[i])
        gradbt = b_grad(fnt[i],y,bj[i])
        gradalphat = alpha_grad(fpt[i],fnt[i],y,alphaj[i])
        aj[i] = aj[i] - eta*(gradat/(N+1)+gamma*(aj[i]-bat[i]))
        bj[i] = bj[i] - eta*(gradbt/(N+1)+gamma*(bj[i]-bbt[i]))
        alphaj[i] = alphaj[i] + eta*gradalphat/(N+1)
    # hessian = hesswt*np.outer(x,x)
    # eigen,_ = np.linalg.eig(hessian)
    
    # print('minimum eigenvalue: %f' %(np.min(eigen)))
    wj = wj - eta*(gradwt*x*y/(N+1) + lam*wj + gamma*(wj - bwt))
    wj = proj(wj,L/2)
    #aJ = proj(aJ,1)
    #bJ = proj(bJ,1)
    #alphaJ = proj(alphaJ,1)
    
    return wj,aj,bj,alphaj

In [16]:
def PGSPD(t,loss,X,Y,L,gamma,lam,theta,c,bwt,bat,bbt,balphat):
    '''
    Proximally Guided Stochastic Primal Dual Algorithm
    '''
    
    # initialize inner loop variables
    Wt = bwt+0.0
    At = bat+0.0
    Bt = bbt+0.0
    ALPHAt = balphat+0.0
    
    BWt = Wt+0.0
    BAt = At+0.0
    BBt = Bt+0.0
    BALPHAt = ALPHAt+0.0
    
    ETAt = c/(t**theta)
    # inner loop update at j
    for j in range(t): 
        # update inner loop variables
        Wt,At,Bt,ALPHAt = prox(ETAt,loss,X[j],Y[j],L,gamma,lam,Wt,At,Bt,ALPHAt,bwt,bat,bbt,balphat)
        BWt += Wt
        BAt += At
        BBt += Bt
        BALPHAt += ALPHAt
        
    # update outer loop variables
    bwt = BWt/t
    bat = BAt/t
    bbt = BBt/t
    balphat = BALPHAt/t
    
    return bwt,bat,bbt,balphat

In [17]:
def SOLAM(t,loss,batch,X,Y,L,lam,theta,c,wt,at,bt,alphat):
    '''
    Stochastic Online AUC Maximization step
    
    input:
        T - total number of iteration
        F - objective function value
        loss - loss function
        pt - p at t
        wt - w at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        W - record of each wt
        A - record of each at
        B - record of each bt
        ALPHA - record of each alphat
    '''
    # Loop in the batch
    peta = c/(t**theta)
    deta = sqrt(log(T*(T+1)/2/batch)/(T*(T+1)/2/batch))
    for k in range(batch):
        
        # Update wt,at,bt
        prod = np.dot(wt,X[k])
        fpt = np.zeros(N+1)
        gfpt = np.zeros(N+1)
        fnt = np.zeros(N+1)
        gfnt = np.zeros(N+1)
        gradwt = 0.0
        gradat = 0.0
        gradbt = 0.0
        gradalphat = 0.0
        
        for i in range(N+1): # add up info of each i
            fpt[i],gfpt[i] = pos(i,prod,L) # partial info
            fnt[i],gfnt[i] = neg(loss,i,prod,L)
            gradwt += w_grad(gfpt[i],gfnt[i],Y[k],at[i],bt[i],alphat[i])
            gradat = a_grad(fpt[i],Y[k],at[i])
            gradbt = b_grad(fnt[i],Y[k],bt[i])
            gradalphat = alpha_grad(fpt[i],fnt[i],Y[k],alphat[i])
            at[i] -= deta*gradat/(N+1)/batch
            bt[i] -= deta*gradbt/(N+1)/batch
            alphat[i] += deta*gradalphat/(N+1)/batch
        
        wt = wt - peta*(gradwt*Y[k]*X[k]/(N+1)/batch + lam*wt) # step size as 1/t gradient descent
        
    wt = proj(wt,L/2)    
        
    return wt,at,bt,alphat

In [26]:
def demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg,gamma=0.1,lam=1.0,theta=0.1,c = 1.0):
    '''
    Run it to get results
    '''
    # define loss function
    if loss == 'hinge':
        L = 2*sqrt(2/lam)
        loss = lambda x: max(0,1+L-2*L*x)
    elif loss == 'logistic':
        L = 2*sqrt(2*log(2)/lam)
        loss = lambda x:log(1+exp(L-2*L*x))
    else:
        print('Wrong loss function!')
        return
    
    # get dimensions of the data
    num,d = X_train_augmented.shape
    
    # initialize outer loop variables
    WT = np.zeros(d) # d is the dimension of the features
    AT = np.zeros(N+1)
    BT = np.zeros(N+1)
    ALPHAT = np.zeros(N+1)

    # record auc
    roc_auc = np.zeros(T)
    # record time elapsed
    start_time = time.time()
    
    for t in range(1,T+1):    
        if alg == 'PGSPD':
        
            if t<num:
                begin = (t*(t-1)//2)%num
                end = (t*(t+1)//2)%num
                if begin < end:
                    x_train = X_train_augmented[begin:end]
                    y_train = Y_train_augmented[begin:end]
                else: # need to think better
                    x_train = np.append(X_train_augmented[begin:],X_train_augmented[:end],axis=0)
                    y_train = np.append(Y_train_augmented[begin:],Y_train_augmented[:end],axis=0)
                # print(sum(x_train))
                x_train, y_train = shuffle(x_train,y_train)
                # update outer loop variables
                WT,AT,BT,ALPHAT = PGSPD(t,loss,x_train,y_train,L,gamma,lam,theta,c,WT,AT,BT,ALPHAT)
            else:
                x_train, y_train = shuffle(X_train_augmented,Y_train_augmented)
                WT,AT,BT,ALPHAT = PGSPD(num,loss,x_train,y_train,L,gamma,lam,theta,c,WT,AT,BT,ALPHAT)
                
        elif alg == 'SOLAM':

            # sample a point
            begin = (t-1)*batch%num
            end = t*batch%num
            if begin < end:
                x_train = X_train_augmented[begin:end]
                y_train = Y_train_augmented[begin:end]
            else: # need to think better
                x_train = np.append(X_train_augmented[begin:],X_train_augmented[:end],axis=0)
                y_train = np.append(Y_train_augmented[begin:],Y_train_augmented[:end],axis=0)
            WT,AT,BT,ALPHAT = SOLAM(t,loss,batch,x_train,y_train,L,lam,theta,c,WT,AT,BT,ALPHAT)
        
        else:
            print('Wrong algorithm!')
            return
    
        fpr, tpr, _ = roc_curve(Y_test, np.dot(X_test,WT))
        roc_auc[t-1] = auc(fpr, tpr)
        if t%100 == 0:
            elapsed_time = time.time() - start_time
            print('gamma: %.1f lam: %.1f theta: %.1f c: %.1f iteration: %d AUC: %.4f time eplapsed: %.2f' 
                  %(gamma,lam,theta,c,t,roc_auc[t-1],elapsed_time))
            start_time = time.time()
    
            
    return WT,AT,BT,ALPHAT,roc_auc

In [19]:
def cv(dataset,loss,alg,folders,gamma=0.0,lam=0.0,theta=1.0,c=1.0):
    '''
    Cross validation
    '''
    # Load data set
    FEATURES,LABELS = loader(dataset) 
    
    # record auc
    AUC_ROC = np.zeros(folders)
    
    # cross validation
    for folder in range(folders):
        print('folder = %d' %(folder))
        X_train_augmented,X_test,Y_train_augmented,Y_test = split(FEATURES,LABELS,folder,folders)
        
        _,_,_,_,roc_auc = demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg,gamma=gamma,lam=lam,theta=theta,c=c)
        AUC_ROC[folder] = max(roc_auc)
    print('auc score: %f +/- %f' %(np.mean(AUC_ROC),np.std(AUC_ROC)))
    return AUC_ROC

In [20]:
def single_run(para):
    folder,gamma,lam,theta,c,paras = para
    X_train_augmented,X_test,Y_train_augmented, Y_test,loss,alg = paras
    _,_,_,_,roc_auc = demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg
                           ,gamma=GAMMA[gamma],lam=LAM[lam],theta=THETA[theta],c=C[c])
    return folder,gamma,lam,theta,c, np.max(roc_auc)
    
def gs(dataset,loss,alg,folders,GAMMA=[0.0],LAM=[0.0],THETA=[1.0],C=[1.0]):
    '''
    Grid search! Wuss up fellas?!
    And we are using multiprocessing, fancy!
    '''
    # number of cpu want to use
    num_cpus = 15
    # Load data set
    FEATURES,LABELS = loader(dataset) 
    # record auc
    AUC_ROC = np.zeros((folders,len(GAMMA),len(LAM),len(THETA),len(C)))
    # record parameters
    input_paras = []
    # grid search prepare
    for folder in range(folders):
        X_train_augmented,X_test,Y_train_augmented,Y_test = split(FEATURES,LABELS,folder,folders)
        paras = X_train_augmented,X_test,Y_train_augmented, Y_test,loss,alg
        for gamma,lam,theta,c in product(range(len(GAMMA)),range(len(LAM)),range(len(THETA)),range(len(C))):
            input_paras.append((folder,gamma,lam,theta,c,paras))
    print('how many paras: %d' % len(input_paras))
    # grid search run on multiprocessors
    pool = multiprocessing.Pool(processes=num_cpus)
    results_pool = pool.map(single_run,input_paras)
    pool.close()
    pool.join()
    # save results
    for folder,gamma,lam,theta,c, auc_roc in results_pool:
        AUC_ROC[folder,gamma,lam,theta,c] = auc_roc
    MEAN = np.zeros((len(GAMMA),len(LAM),len(THETA),len(C)))
    STD = np.zeros((len(GAMMA),len(LAM),len(THETA),len(C)))
    for gamma,lam,theta,c in product(range(len(GAMMA)),range(len(LAM)),range(len(THETA)),range(len(C))):    
        MEAN[gamma,lam,theta,c] = np.mean(AUC_ROC[:,gamma,lam,theta,c])
        STD[gamma,lam,theta,c] = np.std(AUC_ROC[:,gamma,lam,theta,c])
    return AUC_ROC,MEAN,STD

In [21]:
N=10
T=2000
batch=1

In [23]:
GAMMA = [0.1]
LAM = [1.0,10.0]
THETA = [0.25,0.5,0.75,1.0]
C = [10.0,100.0]

In [24]:
run_time = time.time()
usps_hinge,usps_hinge_mean,usps_hinge_std = gs('usps','hinge','PGSPD',5,GAMMA,LAM,THETA,C)
print('total elapsed time: %f' %(time.time() - run_time))

how many paras: 80
iteration: 100 AUC: 0.879456 time eplapsed: 36.989216
iteration: 100 AUC: 0.838404 time eplapsed: 41.373804
iteration: 100 AUC: 0.836429 time eplapsed: 41.902583
iteration: 100 AUC: 0.801507 time eplapsed: 42.354053
iteration: 100 AUC: 0.856245 time eplapsed: 43.054623
iteration: 100 AUC: 0.842044 time eplapsed: 43.078292
iteration: 100 AUC: 0.864675 time eplapsed: 42.528008
iteration: 100 AUC: 0.852600 time eplapsed: 42.779291
iteration: 100 AUC: 0.838594 time eplapsed: 43.562439
iteration: 100 AUC: 0.799372 time eplapsed: 43.282868
iteration: 100 AUC: 0.856717 time eplapsed: 43.991413
iteration: 100 AUC: 0.847957 time eplapsed: 43.492809
iteration: 100 AUC: 0.842033 time eplapsed: 46.105535
iteration: 100 AUC: 0.851069 time eplapsed: 45.295066
iteration: 100 AUC: 0.891161 time eplapsed: 45.544091
iteration: 200 AUC: 0.768720 time eplapsed: 57.218038
iteration: 200 AUC: 0.847124 time eplapsed: 56.402246
iteration: 200 AUC: 0.651584 time eplapsed: 64.542506
iteration

iteration: 1100 AUC: 0.573605 time eplapsed: 189.067965
iteration: 1100 AUC: 0.898364 time eplapsed: 187.243573
iteration: 1100 AUC: 0.856454 time eplapsed: 184.833957
iteration: 1100 AUC: 0.853955 time eplapsed: 196.893878
iteration: 1100 AUC: 0.824177 time eplapsed: 193.618827
iteration: 1100 AUC: 0.844606 time eplapsed: 181.090309
iteration: 1100 AUC: 0.616435 time eplapsed: 192.609183
iteration: 1100 AUC: 0.914196 time eplapsed: 202.392944
iteration: 1100 AUC: 0.720914 time eplapsed: 195.058757
iteration: 1100 AUC: 0.856990 time eplapsed: 204.090300
iteration: 1100 AUC: 0.657650 time eplapsed: 200.741886
iteration: 1100 AUC: 0.270385 time eplapsed: 197.639117
iteration: 1100 AUC: 0.301777 time eplapsed: 189.739588
iteration: 1100 AUC: 0.899370 time eplapsed: 198.627537
iteration: 1100 AUC: 0.894290 time eplapsed: 202.275318
iteration: 1200 AUC: 0.891086 time eplapsed: 194.014426
iteration: 1200 AUC: 0.797819 time eplapsed: 202.112283
iteration: 1200 AUC: 0.776759 time eplapsed: 199

/home/neyo/PycharmProjects/AUC/venv/lib/python3.5/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in double_scalars
/home/neyo/PycharmProjects/AUC/venv/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

iteration: 100 AUC: 0.877416 time eplapsed: 15.031946
iteration: 2000 AUC: 0.578624 time eplapsed: 321.224959
iteration: 2000 AUC: 0.904158 time eplapsed: 331.695915


...

iteration: 100 AUC: 0.773422 time eplapsed: 24.678809
iteration: 2000 AUC: 0.654841 time eplapsed: 405.044140
iteration: 2000 AUC: 0.328991 time eplapsed: 388.430632
iteration: 300 AUC: 0.899193 time eplapsed: 84.060402
iteration: 100 AUC: 0.821152 time eplapsed: 28.564067
iteration: 300 AUC: 0.870160 time eplapsed: 72.525721
iteration: 100 AUC: 0.865566 time eplapsed: 27.148165
iteration: 400 AUC: 0.869614 time eplapsed: 88.019900
iteration: 100 AUC: 0.898135 time eplapsed: 25.430711
iteration: 100 AUC: 0.524965 time eplapsed: 26.823241
iteration: 300 AUC: 0.784053 time eplapsed: 90.407627
iteration: 200 AUC: 0.885831 time eplapsed: 51.966971
iteration: 300 AUC: 0.902572 time eplapsed: 92.014251
iteration: 200 AUC: 0.896955 time eplapsed: 60.692423
iteration: 300 AUC: 0.856934 time eplapsed: 93.806286
iteration: 200 AUC: 0.902876 time eplapsed: 56.176126
iteration: 400 AUC: 0.854765 time eplapsed: 111.939065
iteration: 200 AUC: 0.909298 time eplapsed: 54.355729
iteration: 200 AUC: 0.8

iteration: 1200 AUC: 0.778759 time eplapsed: 206.328303
iteration: 1200 AUC: 0.892914 time eplapsed: 206.679784
iteration: 1200 AUC: 0.881765 time eplapsed: 208.313877
iteration: 1200 AUC: 0.713594 time eplapsed: 205.785928
iteration: 1200 AUC: 0.882982 time eplapsed: 207.417614
iteration: 1200 AUC: 0.849796 time eplapsed: 208.674839
iteration: 1300 AUC: 0.903326 time eplapsed: 229.445761
iteration: 1200 AUC: 0.863236 time eplapsed: 200.883970
iteration: 1200 AUC: 0.891344 time eplapsed: 213.402394
iteration: 1200 AUC: 0.886898 time eplapsed: 207.226332
iteration: 1300 AUC: 0.883928 time eplapsed: 218.987850
iteration: 1300 AUC: 0.905150 time eplapsed: 213.029779
iteration: 1300 AUC: 0.725798 time eplapsed: 223.554361
iteration: 1300 AUC: 0.481298 time eplapsed: 207.781207
iteration: 1300 AUC: 0.909435 time eplapsed: 227.209880
iteration: 1300 AUC: 0.799499 time eplapsed: 229.591065
iteration: 1300 AUC: 0.902802 time eplapsed: 219.549040
iteration: 1300 AUC: 0.874077 time eplapsed: 236

Process ForkPoolWorker-12:
Process ForkPoolWorker-15:
Process ForkPoolWorker-1:
Process ForkPoolWorker-14:
Process ForkPoolWorker-5:
Process ForkPoolWorker-13:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-6:
Traceback (most recent call last):
Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-11:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-15-4097c1d9a577>", line 20, in prox
    fnt[i],gfnt[i] = neg(loss,i,prod,L)
  File "<ipython-input-18-fd22d75dc72b>", line 45, in demo
    WT,AT,BT,ALPHAT = PGSPD(t,loss,x_train,y_train,L,gamma,lam,theta,c,WT,AT,BT,ALPHAT)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-20-84100d25d341>", line 5, in single_run
    ,gamma=GAMMA[gamma],lam=LAM[lam],theta=THETA[theta],c=C[c])
  File "<ipython-input-20-84100d25d341>", line 5, in single_run
    ,gamma=GAMMA[gamma],lam=LAM[lam],theta=THETA[theta],c=C[c])
  File "<ipython-input-16-8fe792301771>", line 21, in PGSPD
    Wt,At,Bt,ALPHAt = prox(ETAt,loss,X[j],Y[j],L,gamma,lam,Wt,At,Bt,ALPHAt,bwt,bat,bbt,balphat)
  File "<ipython-input-20-

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-15-4097c1d9a577>", line 20, in prox
    fnt[i],gfnt[i] = neg(loss,i,prod,L)
KeyboardInterrupt
  File "<ipython-input-12-408fd4f896d3>", line 7, in alpha_grad
    gradalphat = -2*(alphat - fpt)
KeyboardInterrupt
  File "<ipython-input-6-418faf4458ea>", line 25, in neg
    delta += comb_dict[k][j]*(-1)**(k-j)*loss(j/N)
KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-6-418faf4458ea>", line 25, in neg
    delta += comb_dict[k][j]*(-1)**(k-j)*loss(j/N)
KeyboardInterrupt
  File "<ipython-input-18-fd22d75dc72b>", line 45, in demo
    WT,AT,BT,ALPHAT = PGSPD(t,loss,x_train,y_train,L,gamma,lam,theta,c,WT,AT,BT,ALPHAT)
KeyboardInterrupt
  File "<ipython-input-16-8fe792301771>", line 21, in PGSPD
    Wt,At,Bt,ALPHAt = prox(ETAt,loss,X[j],Y[j],L,gamma,lam,Wt,At,Bt,ALPHAt,bwt,bat,bbt,balphat)
  File "<ipython-input-15-4097c1d9a577>", line 33, in prox
    wj = wj - eta*(gradwt*x*y/(N+1) + lam*wj + gamma*(wj - bwt

In [25]:
usps_hinge_mean

NameError: name 'usps_hinge_mean' is not defined